In [1]:
#Let's first import required Python packages.
#Importing required Python packages
import matplotlib.pylab as plt
import numpy as np
from sklearn.model_selection import ShuffleSplit, train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier  
from sklearn.linear_model import LogisticRegression

import pandas as pd
np.random.seed(42)

/Users/rborra/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
traincsv=pd.read_csv('train.csv', index_col='Id')
testcsv=pd.read_csv('test.csv', index_col='Id')

In [3]:
#80/20 Split
X_train, X_test, y_train, y_test = train_test_split(traincsv.iloc[:,:-1].values, traincsv.iloc[:,-1:].values.ravel(), 
                                                    test_size=0.2)
print X_train.shape, X_test.shape, y_train.shape, y_test.shape

(12096, 54) (3024, 54) (12096,) (3024,)


In [4]:
RFC=RandomForestClassifier(max_depth=14, n_estimators=1000)
RFC.fit(X_train, y_train)

ETC= ExtraTreesClassifier(max_depth=14, n_estimators=1000)
ETC.fit(X_train, y_train)

GBC= GradientBoostingClassifier(max_depth=14, n_estimators=1000)
GBC.fit(X_train, y_train)

XGB= XGBClassifier(max_depth=14, n_estimators=1000)
XGB.fit(X_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=14,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [5]:
estimators = [RFC, ETC, GBC,XGB]

X_blend_train=[]
X_blend_test=[]
X_blend_testcsv=[]

for i, est in enumerate(estimators):
    print i , est
    X_blend_train.append(est.predict(X_train))
    X_blend_test.append(est.predict(X_test))
    X_blend_testcsv.append(est.predict(testcsv.values))

0 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=14, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)
1 ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=14, max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=1000, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)
2 GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=14,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_

In [6]:
X_blend_train = np.array(X_blend_train).T
X_blend_test = np.array(X_blend_test).T
X_blend_testcsv = np.array(X_blend_testcsv).T

In [14]:
from sklearn.linear_model import LogisticRegression 
#lo=LogisticRegression().fit(X_blend_train, y_train) 
#y_pred=lo.predict(X_blend_test)

XGB2= XGBClassifier(max_depth=14, n_estimators=1000).fit(X_blend_train, y_train) 
y_pred=XGB2.predict(X_blend_test)

#Make a copy of the test.csv 
temp=testcsv.copy()

#Run Predictions on test.csv 
temp['Cover_Type']=XGB2.predict(X_blend_testcsv) 

#Create Submissions csv file 
temp=temp['Cover_Type'] 
temp.to_csv('SampleSubmission11', header=True) 

In [ ]:
# Public Score : 0.74605